<a href="https://colab.research.google.com/github/yankeesong/2021-CS109A/blob/master/NerfStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@markdown Install Nerfstudio and Dependencies (~8 min)

%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# # Installing COLMAP
# %cd /content/
# !sudo apt-get install \
#     git \
#     cmake \
#     ninja-build \
#     build-essential \
#     libboost-program-options-dev \
#     libboost-filesystem-dev \
#     libboost-graph-dev \
#     libboost-system-dev \
#     libboost-test-dev \
#     libeigen3-dev \
#     libflann-dev \
#     libfreeimage-dev \
#     libmetis-dev \
#     libgoogle-glog-dev \
#     libgflags-dev \
#     libsqlite3-dev \
#     libglew-dev \
#     qtbase5-dev \
#     libqt5opengl5-dev \
#     libcgal-dev \
#     libceres-dev
# !sudo apt-get install -y libgtest-dev cmake
# !cd /usr/src/gtest && sudo cmake CMakeLists.txt && sudo make

# !sudo apt-get install gcc-10 g++-10
# !export CC=/usr/bin/gcc-10
# !export CXX=/usr/bin/g++-10
# !export CUDAHOSTCXX=/usr/bin/g++-10

# COLMAP
# !export CUDA_PATH=/usr/local/cuda
# !export CPLUS_INCLUDE_PATH=$CUDA_PATH/include
# !export CMAKE_CXX_FLAGS="-I$CUDA_PATH/include"
# !export CMAKE_EXE_LINKER_FLAGS="-L$CUDA_PATH/lib64"
# # !git clone https://github.com/colmap/colmap.git
# %cd /content/
# %cd colmap
# !git checkout dev
# # !mkdir build
# %cd build
# !cmake -DCMAKE_CUDA_ARCHITECTURES='native' .. -GNinja
# !ninja
# !sudo ninja install

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
/content
Downloading...
From: https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t
To: /content/tinycudann-1.7-cp310-cp310-linux_x86_64.whl
100% 31.2M/31.2M [00:00<00:00, 148MB/s]
Processing ./tinycudann-1.7-cp310-cp310-linux_x86_64.whl
/content
  Cloning https://github.com/nerfstudio-project/nerfstudio.git to /tmp/pip-req-build-qurpebbs
  Running command git clone --filter=blob:none --quiet https://github.com/nerfstudio-project/nerfstudio.git /tmp/pip-req-build-qurpebbs
  Resolved https://github.com/nerfstudio-project/nerfstudio.git to commit ce6b9bdd5eeec18e783ff9d65c9a716e91530dc4
  Installing build dependencies ... done
  Getting requirements to bu

In [2]:
# #@markdown  Downloading and Processing Data
# #@markdown Pick the preset scene or upload your own images/video
# import os
# import glob
# from google.colab import files
# from IPython.core.display import display, HTML

# scene = '\uD83D\uDCE4 upload your images' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
# scene = ' '.join(scene.split(' ')[1:])

# if scene == "upload Polycam data":
#     %cd /content/
#     !mkdir -p /content/data/nerfstudio/custom_data
#     %cd /content/data/nerfstudio/custom_data/
#     uploaded = files.upload()
#     dir = os.getcwd()
#     if len(uploaded.keys()) > 1:
#         print("ERROR, upload a single .zip file when processing Polycam data")
#     dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
#     !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
#     scene = "custom_data"
# elif scene == 'upload your own Record3D data':
#     display(HTML('Zip your Record3D folder, and upload.'))
#     display(HTML('More information on Record3D can be found here.'))
#     %cd /content/
#     !mkdir -p /content/data/nerfstudio/custom_data
#     %cd /content/data/nerfstudio/custom_data/
#     uploaded = files.upload()
#     dir = os.getcwd()
#     preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
#     record_3d_zipfile = preupload_datasets[0]
#     !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
#     custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
#     !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
#     scene = "custom_data"
# elif scene in ['upload your images', 'upload your own video']:
#     display(HTML('Select your custom data'))
#     display(HTML('You can select multiple images by pressing ctrl, cmd or shift and click.'))
#     display(HTML('Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.'))
#     !mkdir -p /content/data/nerfstudio/custom_data
#     if scene == 'upload your images':
#         !mkdir -p /content/data/nerfstudio/custom_data/raw_images
#         %cd /content/data/nerfstudio/custom_data/raw_images
#         uploaded = files.upload()
#         dir = os.getcwd()
#     else:
#         %cd /content/data/nerfstudio/custom_data/
#         uploaded = files.upload()
#         dir = os.getcwd()
#     preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
#     del uploaded
#     %cd /content/

#     if scene == 'upload your images':
#         !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
#     else:
#         video_path = preupload_datasets[0]
#         !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

#     scene = "custom_data"
# else:
#     %cd /content/
#     !ns-download-data nerfstudio --capture-name=$scene

# print("Data Processing Succeeded!")

In [3]:
# !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/

In [4]:
#@markdown Set up and Start Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.231s
https://viewer.nerf.studio/?websocket_url=wss://witty-nails-lie.loca.lt
You may need to click Refresh Page after you start training!


In [6]:
#@markdown Start Training
import os
%cd /content
if os.path.exists(f"penguin/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data penguin --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('Error: Data processing did not complete'))
    display(HTML('Please re-run `Downloading and Processing Data`, or view the FAQ for more info.'))

流式输出内容被截断，只能显示最后 5000 行内容。
26510 (88.37%)      63.266 ms            3 m, 40 s            65.99 K                                
26520 (88.40%)      64.683 ms            3 m, 45 s            64.19 K                                
26530 (88.43%)      64.367 ms            3 m, 43 s            64.49 K                                
26540 (88.47%)      66.605 ms            3 m, 50 s            62.58 K                                
---------------------------------------------------------------------------------------------------- 
Viewer at: https://viewer.nerf.studio/versions/23-05-15-1/?websocket_url=ws://localhost:7007         
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
26460 (88.20%)      56.424 ms            3 m, 19 s            73.68 K                                
26470 (88.23%)      56.918 ms            3 m, 20 s     

In [9]:
# Now instructNerf2Nerf
# %cd /content/
# !git clone https://github.com/ayaanzhaque/instruct-nerf2nerf.git
# %cd instruct-nerf2nerf
# !pip install --upgrade pip setuptools
# !pip install -e .

2023-07-31 08:04:16.509793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
%cd /content/
!ns-train in2n --data penguin --load-dir {outputs/unnamed/nerfacto/2023-07-31_071328/nerfstudio_models} --pipeline.prompt {"give the penguin a red hat"} --pipeline.guidance-scale 7.5 --pipeline.image-guidance-scale 1.5


/content
2023-07-31 08:09:24.744977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[08:09:33] Using --data alias for --data.pipeline.datamanager.data                                          ]8;id=524766;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/train.py\train.py]8;;\:]8;id=157438;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/train.py#231\231]8;;\
──────────────────────────────────────────────────────── Config ────────────────────────────────────────────────────────
InstructNeRF2NeRFTrainerConfig(
    _target=<class 'in2n.in2n_trainer.InstructNeRF2NeRFTrainer'>,
    output_dir=PosixPath('outputs'),
    method_name='in2n',
    experiment_name=None,
    project_name='nerfstudio-project',
    timestamp='2023-07-31_080933',
    machine=MachineConfig(seed=42, num_devices=1, num_machines=1, machine_rank=0, dist_url='auto', device_type='cuda'),
    logging=LoggingConfig(
        relative_log_